In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix
import nltk
from nltk import word_tokenize
import re
from sklearn import feature_extraction, feature_selection, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
data = pd.read_excel(r"C:\Users\gog10\GfK_spectacle_lenses_data.xlsx")

In [3]:
#Replace null values in the data
data.loc[data['main_text'].isnull(), 'main_text'] = 'Not Known'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3549 entries, 0 to 3548
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   item_id         3549 non-null   int64 
 1   main_text       3549 non-null   object
 2   country_name    3549 non-null   object
 3   retailer_pg     3549 non-null   object
 4   mdm_brand_text  3549 non-null   object
 5   mdm_model_text  3549 non-null   object
dtypes: int64(1), object(5)
memory usage: 166.5+ KB


In [4]:
#Get counts for item id
data['mdm_brand_text'].value_counts()

ESSILOR           1266
HOYA               651
ZEISS              421
INDO               292
RODENSTOCK         279
SYNCHRONY          164
RUPP & HUBRACH     127
MAILSHOP            81
PRATS               57
SHAMIR              42
MEDICAL VISION      30
NIKA                29
KODAK               19
SEIKO               15
NIKON               11
RAY BAN             10
EYEART              10
TOKAI                9
WETZLICH             9
AMERICAN OPTIC       7
L.O.A.               7
BBGR                 7
VISALL               6
Name: mdm_brand_text, dtype: int64

In [5]:
#Delete columns not needed for model
data.drop(['item_id','country_name', 'mdm_model_text'], axis = 'columns', inplace=True)

In [6]:
data.head()

,main_text,retailer_pg,mdm_brand_text
0,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
1,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
2,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
3,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
4,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA


In [7]:
## rename columns
data = data.rename(columns={"mdm_brand_text":"y", "main_text":"text", "retailer_pg":"pg"})
data

,text,pg,y
0,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
1,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
2,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
3,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
4,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA
...,...,...,...
3544,CRISTALES 34212 MI MF XPERIENCE 1.5 BASIC HV P...,CRISTALES,HOYA
3545,CRISTALES 34186 MI HILUX 1.6 SHV FAB MONOFOCAL...,CRISTALES,HOYA
3546,CRISTALES 34317 NULUX ACTIVE TF 1.5 HV ORGÁNIC...,CRISTALES,HOYA
3547,CRISTALES 34244 MI LIFESTYLE 3I 1.5 HVLL BC PR...,CRISTALES,HOYA


In [8]:
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [10]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def preprocess_pg(pg, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (remove punctuations and characters and then strip)
    pg = re.sub(r'[^\w\s]', '', str(pg).strip())
            
    ## Tokenize (convert from string to list)
    lst_pg = pg.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_pg = [word for word in lst_pg if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_pg = [ps.stem(word) for word in lst_pg]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_pg = [lem.lemmatize(word) for word in lst_pg]
            
    ## back to string from list
    pg = " ".join(lst_pg)
    return pg

In [11]:
data["text_clean"] = data["text"].apply(lambda x: 
          preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
data.head()

,text,pg,y,text_clean
0,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX
1,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX
2,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX
3,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX
4,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX


In [12]:
data["pg_clean"] = data["pg"].apply(lambda x: 
          preprocess_pg(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
data.head()

,text,pg,y,text_clean,pg_clean
0,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX,CRISTALES
1,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX,CRISTALES
2,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX,CRISTALES
3,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX,CRISTALES
4,ID1.60LIFESTL3IURBAN(RX),CRISTALES,HOYA,ID160LIFESTL3IURBANRX,CRISTALES


In [13]:
X = data.text_clean + data.pg_clean
y = data['y']


In [14]:
## split dataset
data_train, data_test = model_selection.train_test_split(data, test_size=0.3)
## get target
y_train = data_train["y"].values
y_test = data_test["y"].values

In [15]:
## Count (classic BoW)
vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,2))

## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

In [16]:
texts = data_train["text_clean"]
vectorizer.fit(texts)
X_train = vectorizer.transform(texts)
dic_vocabulary = vectorizer.vocabulary_

In [17]:
vectorizer.get_feature_names()

['000',
 '000 0000',
 '000 ess',
 '000 loa',
 '0000',
 '0000 0000',
 '0000 0050',
 '0000 0075',
 '0000 0100',
 '0000 0125',
 '0000 0150',
 '0000 0175',
 '0000 0200',
 '0000 0225',
 '0000 0250',
 '0000 0275',
 '0000 0300',
 '0000 0325',
 '0000 0350',
 '0000 200',
 '0000 2250',
 '0000 300',
 '0000 airwear',
 '0000 varilux',
 '0002',
 '000202',
 '000202 masterlux',
 '001101',
 '001101 hifin',
 '001102',
 '001102 hifin',
 '0016',
 '0025',
 '0025 0100',
 '0025 0125',
 '0025 0150',
 '0025 0225',
 '0025 0250',
 '0025 0275',
 '0025 0300',
 '0025 0350',
 '0025 250',
 '0025 airwear',
 '0025 ormix',
 '004',
 '004 0125',
 '005',
 '005 0075',
 '005 0125',
 '005 0150',
 '005 0225',
 '0050',
 '0050 0000',
 '0050 0025',
 '0050 0050',
 '0050 0075',
 '0050 0100',
 '0050 0150',
 '0050 0175',
 '0050 0225',
 '0050 0275',
 '0050 0300',
 '0050 0375',
 '0050 0400',
 '0050 0475',
 '0050 0550',
 '0050 0750',
 '0050 125',
 '0050 200',
 '0050 250',
 '0050 airwear',
 '0050 aiwear',
 '0050 crizal',
 '0050 ormix',
 

In [18]:
vectorizer.vocabulary_

{'nikacolor': 5011,
 '15': 606,
 'inpro2': 4383,
 'polarized': 5428,
 'hard': 4015,
 'rkset': 5681,
 'nikacolor 15': 5012,
 '15 inpro2': 642,
 'inpro2 polarized': 4384,
 'polarized hard': 5429,
 'hard rkset': 4021,
 'rod': 5703,
 'i3gh': 4218,
 'su': 6048,
 'rod i3gh': 5709,
 'i3gh rod': 4220,
 'rod su': 5724,
 'lente': 4477,
 'varilux': 6381,
 'xclusive': 6507,
 '4d': 1719,
 'stylis': 6044,
 '167': 931,
 'eps': 3454,
 'organica': 5141,
 'progresiva': 5536,
 'lente varilux': 4551,
 'varilux xclusive': 6397,
 'xclusive 4d': 6509,
 '4d stylis': 1725,
 'stylis 167': 6045,
 '167 eps': 958,
 'eps organica': 3485,
 'organica progresiva': 5144,
 'flkft': 3838,
 'rod flkft': 5707,
 'flkft rod': 3840,
 'ormix': 5210,
 '16': 737,
 'crizal': 3086,
 'prevencia': 5482,
 'ormix 16': 5211,
 '16 crizal': 773,
 'crizal prevencia': 3097,
 'cristales': 2890,
 '494': 1712,
 'hil': 4055,
 '01': 92,
 'stk': 6006,
 'org': 5117,
 '160': 870,
 'shv': 5893,
 'monofocal': 4910,
 'orgánico': 5168,
 'multicapa': 4

In [19]:
array_of_feature = X_train.toarray()
array_of_feature

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
y = data_train["y"]
X_names = vectorizer.get_feature_names()
p_value_limit = 0.95
data_features = pd.DataFrame()
for cat in np.unique(y):
    chi2, p = feature_selection.chi2(X_train, y==cat)
    data_features = data_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    data_features = data_features.sort_values(["y","score"], 
                    ascending=[True,False])
    data_features = data_features[data_features["score"]>p_value_limit]
X_names = data_features["feature"].unique().tolist()

In [21]:
for cat in np.unique(y):
   print("# {}:".format(cat))
   print("  . selected features:",
         len(data_features[data_features["y"]==cat]))
   print("  . top features:", ",".join(
data_features[data_features["y"]==cat]["feature"].values[:10]))
   print(" ")

# AMERICAN OPTIC:
  . selected features: 27
  . top features: 16 hcm,16 photofg,16 photomarron,7075 bas,american,american optical,bas,hcm,hcm zeiss,hmc 6570
 
# BBGR:
  . selected features: 19
  . top features: 17,17 60,17 65,17 70,17 diams,17 stock,60 diams,65 diams,70 diams,diams
 
# ESSILOR:
  . selected features: 295
  . top features: crizal,ormix,ormix 16,prevencia,ess,crizal prevencia,varilux,16 crizal,eps,orma
 
# EYEART:
  . selected features: 30
  . top features: 16 ultra,161,161 antirreflejo,161 hmc,174 super,2540,2540 orlite,2541,2541 orlite,33997
 
# HOYA:
  . selected features: 281
  . top features: 160,eyas,hilux,id160lifestl3iurbanrx,lifestyle,rx,id,balansis,hilux eyas,indoor
 
# INDO:
  . selected features: 468
  . top features: activa_rvd,basic,durcap,indosol,indosol basic,natural10,superfin,indofin,indofin 16,basic gris
 
# KODAK:
  . selected features: 59
  . top features: 13,13 15,13 174,14 15,15 15,15 clear,15 trans,16 174,161 kodak,174 clear
 
# L.O.A.:
  . select

In [22]:
classifier = naive_bayes.MultinomialNB()

## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train)
## test
X_test = data_test["text_clean"].values
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

In [23]:
classes = np.unique(y_test)
y_test_array = pd.get_dummies(y_test, drop_first=False).values

precision = precision_score(y_test, predicted, average='micro', zero_division=0)
print('Precision score: {0:0.2f}'.format(precision))

recall = recall_score(y_test, predicted, average='micro', zero_division=0)
print('Recall score: {0:0.2f}'.format(recall))

f1 = f1_score(y_test, predicted, average='micro', zero_division=0)
print('f1 score: {0:0.2f}'.format(recall))

Precision score: 0.88
Recall score: 0.88
f1 score: 0.88
